# FOOD CLASSIFICATION PROJECT

*Pytorch project for AI Engineering Master with ProfessionAI*

## Colab Configuration

If you're testing this project on Google Colab, it could be useful for you to run the following cells

In [ ]:
# If you are running this code on Google Colab, run this:

!git clone https://github.com/Silvano315/PyTorch-CNN-for-food-image-classification-system.git

%cd PyTorch-CNN-for-food-image-classification-system
%pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import Libraries and initial Set-Up

In [1]:
# Import libraries

import os
import sys
import json
import random
import logging
import time

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.constants import RANDOM_SEED, DATA_PATH, BATCH_SIZE
from src.utils import get_paths_to_files, get_dataset_paths, get_logger
from src.viz_fx import display_random_images, visualize_class_samples, plot_class_distribution, compare_class_distribution, \
                         analyze_image_dimensions, analyze_color_distribution, visualize_random_images, visualize_augmented_images
from src.preprocessing import create_datasets, create_data_loaders
from src.models import create_model, Experiment, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, train_model, validate

In [2]:
# RUN THIS CELL IF AND ONLY IF YOU'RE RUNNING THIS REPOSITORY ON GOOGLE COLAB

# set your path to your dataset on Google Drive
DATA_PATH =  "/content/drive/MyDrive/Project_PyTorch_ProfessionAI/dataset/"

In [2]:
# Set Random Seed for reproducibility

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
# Check if GPU is available and set it as device 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This repository is connected to {str(device).upper()}")

This repository is connected to CPU


## Exploratory Data Analysis

In [4]:
# Get paths and filenames form directory dataset 

filepaths, filenames = get_paths_to_files(DATA_PATH)

print("="*80)
print("File paths:")
for i in range(0,5):
    print(filepaths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(filenames[i])
print("="*80)

File paths:
dataset/test/Sandwich/Sandwich-Train (36)_e50a3a403c2741408f47d8bda7e28d1f.jpeg
dataset/test/Sandwich/Sandwich-Train (465)_f1e4e45cc2c846f19b6e2faefe8f4600.jpeg
dataset/test/Sandwich/Sandwich-Train (1162)_8f665882c03c496eba53724221b04846.jpeg
dataset/test/Sandwich/Sandwich-Train (578)_3b7f8599e1b4472f9db6fa7666eae82c.jpeg
dataset/test/Sandwich/Sandwich-Train (717)_4603bb6b236e48e9974d6b07dce7157c.jpeg
File names:
Sandwich-Train (36)_e50a3a403c2741408f47d8bda7e28d1f.jpeg
Sandwich-Train (465)_f1e4e45cc2c846f19b6e2faefe8f4600.jpeg
Sandwich-Train (1162)_8f665882c03c496eba53724221b04846.jpeg
Sandwich-Train (578)_3b7f8599e1b4472f9db6fa7666eae82c.jpeg
Sandwich-Train (717)_4603bb6b236e48e9974d6b07dce7157c.jpeg


In [5]:
# Access to path dirs and file names for each split (test, train, val)

dataset_paths = get_dataset_paths(DATA_PATH)

train_paths, train_names = dataset_paths['train']
test_paths, test_names = dataset_paths['test']
val_paths, val_names = dataset_paths['val']

print("="*80)
print("File paths:")
for i in range(0,5):
    print(train_paths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(train_names[i])
print("="*80)

File paths:
dataset/train/Sandwich/Sandwich-Train (277)_436fb6c2c1fc4e91a9be68c03e8e45a4.jpeg
dataset/train/Sandwich/Sandwich-Train (161)_02ffbe626d9c4d969a256034e0617fdc.jpeg
dataset/train/Sandwich/Sandwich-Train (1368)_627dc1a3443d4750bafe0ca7d446ed0b.jpeg
dataset/train/Sandwich/Sandwich-Train (1016)_b9a1f780c2b54c4a8af769240dfbddb9.jpeg
dataset/train/Sandwich/Sandwich-Train (318)_778f0227997d4e50b16153d8516e5088.jpeg
File names:
Sandwich-Train (277)_436fb6c2c1fc4e91a9be68c03e8e45a4.jpeg
Sandwich-Train (161)_02ffbe626d9c4d969a256034e0617fdc.jpeg
Sandwich-Train (1368)_627dc1a3443d4750bafe0ca7d446ed0b.jpeg
Sandwich-Train (1016)_b9a1f780c2b54c4a8af769240dfbddb9.jpeg
Sandwich-Train (318)_778f0227997d4e50b16153d8516e5088.jpeg


In [ ]:
# Visualize n random images 

fig, axes = display_random_images(filepaths, n=25)
plt.show() 

In [ ]:
# Visualize n random images from a chosen split dataset (train_paths, test_paths, val_paths)

chosen_split = train_paths

fig, axes = display_random_images(chosen_split, n=25)
plt.show() 

In [ ]:
# Displays sample images for each class in the chosen split set

chosen_split = train_paths

fig = visualize_class_samples(chosen_split, num_samples=3, max_classes=11)
plt.show()

In [6]:
# Bar plot for class distribution for each dataset split

chosen_split = val_paths

class_dist_fig = plot_class_distribution(chosen_split)
class_dist_fig.show()

In [6]:
# Comparison of the Class Distribution between Train, Test e Validation

comparison_fig = compare_class_distribution(dataset_paths)
comparison_fig.show()

In [9]:
# Several plots to analyze Images Dimensions

dim_fig = analyze_image_dimensions(test_paths)
dim_fig.show()


In [ ]:
# Histogram Plots to analyze Colour Distribution for a chosen split dataset

chosen_split = train_paths

color_dist_fig = analyze_color_distribution(chosen_split, n_samples=50)
color_dist_fig.show()

## Preprocessing 

In [4]:
# Preprocessing for Baseline Model (without augmentation, also for train set)
# Resize Dimensions (224,224) and Normalization

trainset, valset, testset = create_datasets(DATA_PATH, augment_train=False)

print('='*60 + "\nTrain:")
print(trainset)
print('='*60 + "\Val:")
print(valset)
print('='*60 + "\Test:")
print(testset)
print("="*60)

Train:
Dataset ImageFolder
    Number of datapoints: 8960
    Root location: dataset//train
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a0ae6c10>
============================================================\Val:
Dataset ImageFolder
    Number of datapoints: 2240
    Root location: dataset//val
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a0f21350>
============================================================\Test:
Dataset ImageFolder
    Number of datapoints: 2800
    Root location: dataset//test
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a0f21350>


In [ ]:
# Visualization of random images from ImageFolder

chosen_folder = trainset

visualize_random_images(chosen_folder, num_images=16, axis=False)

In [7]:
# Preprocessing for Transfer Learning Model (with augmentation for train set)
# Resize Dimensions (224,224) and Normalization

trainset_aug, valset, testset = create_datasets(DATA_PATH, augment_train=True)

print('='*60 + "\nTrain:")
print(trainset_aug)
print('='*60 + "\Val:")
print(valset)
print('='*60 + "\Test:")
print(testset)
print("="*60)

Train:
Dataset ImageFolder
    Number of datapoints: 8960
    Root location: dataset//train
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x1082fdc90>
============================================================\Val:
Dataset ImageFolder
    Number of datapoints: 2240
    Root location: dataset//val
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a2655510>
============================================================\Test:
Dataset ImageFolder
    Number of datapoints: 2800
    Root location: dataset//test
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a2655510>


/Users/silvanoquarto/Desktop/PROJECTS/Master_AI_Engineering/PyTorch-DNN-for-food-image-classification-system/.venv/lib/python3.11/site-packages/pydantic/main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
# Visualization of random augmented images from ImageFolder

visualize_augmented_images(trainset_aug, num_images=5)

In [5]:
# Create DataLoader for each dataset split

is_augmented = False
datasets = {
    'train': trainset_aug if is_augmented else trainset,
    'val': valset,
    'test': testset
}

dataloaders = create_data_loaders(datasets, batch_size=BATCH_SIZE)

train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

## Baseline Model

In [6]:
# Define configurations

num_epochs = 1
learning_rate = 0.001
num_classes = len(trainset.classes)

In [7]:
# Create model and  set to device

logger = get_logger(ch_log_level=logging.INFO, fh_log_level=logging.INFO)

model = create_model(num_classes)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
         MaxPool2d-2         [-1, 32, 112, 112]               0
            Conv2d-3         [-1, 64, 112, 112]          18,496
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]          73,856
         MaxPool2d-6          [-1, 128, 28, 28]               0
 AdaptiveAvgPool2d-7            [-1, 128, 1, 1]               0
           Flatten-8                  [-1, 128]               0
            Linear-9                  [-1, 512]          66,048
          Dropout-10                  [-1, 512]               0
           Linear-11                   [-1, 14]           7,182
Total params: 166,478
Trainable params: 166,478
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/

In [10]:
# Initialize Experiment

experiment = Experiment("BaselineCNN", "experiments")
experiment.init()

In [11]:
# Training step

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=True),
    ModelCheckpoint(filepath='best_model.pth', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
]

since = time.time()
trained_model = train_model(model, train_loader, val_loader, experiment, callbacks, num_epochs, device)
time_elapsed = time.time() - since
logger.info('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
# Plot history

experiment.plot_history()

In [ ]:
# Evaluation on test set

test_logs = validate(trained_model, test_loader, torch.nn.CrossEntropyLoss(), device)
logger.info(f"Test Results: {test_logs}")

In [ ]:
# Save model

torch.save(trained_model.state_dict(), 'final_baseline_model.pth')
logger.info("Final model saved as 'final_model.pth'")

### Model Evaluation

## Transfer Learning

### Model Evaluation